# Transformer疑问以及补充笔记

## 1、Transformer整体架构

### 1.3、一句话是如何变成向量的：

1、分词：将句子分成一个个词汇  
2、数值化：对照自己构建的或用别人预训练的词汇表进行数值化，数值化后通常是一个整数列表  
3、嵌入层转换：使用嵌入层将每一个ID转换为密集向量，输出二维张量（对于一个句子来说）。    
&ensp;&ensp;嵌入层：一个强化版的词汇表，不仅记录有哪些词，还为这些词分配了一个**语义向量（推荐512维，性价比最高）**。  
4、位置编码：为每一个词添加位置信息，transformer没有位置的概念，不添加位置信息，打乱句子顺序后的输出是一样的。一个词向量如果有512维，他的位置信息也有512维。10000是缩放常数，是用来控制正弦波和余弦波的频率范围。对于偶数维度使用sin位置编码，对于奇数维度使用cos位置编码。  
    
$$
PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{\frac{2i}{d_{\text{model}}}}}\right)
$$
$$
PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{\frac{2i}{d_{\text{model}}}}}\right)
$$  
5、嵌入与位置编码相加：将词汇表中每个词查表得到的 512维词嵌入向量，与对应位置通过公式计算出的 512维位置编码向量，进行向量加法，得到最终输入到 Transformer 的表示。  


关于一个问题的解释：既然位置信息都已经加到了词向量中，模型又怎么得知位置的信息呢？  
相加是一种优雅的隐式编码：它让位置信息**渗透**到每个维度，而不是作为一个外部信号。当一个词出现在不同位置时，它的**混合向量**会不同，导致它在上下文中的行为也不同。



## 2、Transformer的q,k,v过程

### 2.2、q,k,v过程

q代表查询，k代表键向量，与查询向量匹配，计算注意力分数，v代表值向量，代表实际内容。  
通过嵌入层得到的向量表示x，并设置权重矩阵Wk,Wq,Wv，这三个矩阵维度是由嵌入维度和注意力头数决定。Wk,Wq,Wv的维度都是 d_model × d_model，但它们**内部**被分成了 num_heads 份，每份对应一个头，大小为 d_model × (d_model // num_heads)。  

**计算q,k,v：**

以x中的x1（第一个词的嵌入维度）举例：  
$$ 
\begin{aligned}
Q_1 = x_1 \cdot Wq \\  
K_1 = x_1 \cdot Wk \\ 
V_1 = x_1 \cdot Wv  
\end{aligned}
$$
这些就是x1的q,k,v

**计算注意力分数:**

需要计算每一个词之间的注意力分数，包括和自己。  
$$
\begin{aligned}
score(Q_1,K_1)=Q_1 \cdot K_1^\top \\
score(Q_1,K_2)=Q_1 \cdot K_2^\top \\
score(Q_1,K_3)=Q_1 \cdot K_3^\top 
\end{aligned}
$$



**进行缩放:**

先明确一个值d_K:$$d_k = \frac{d_{\text{model}}}{\text{num_heads}}$$  
num_heads是注意力头数量。  
缩放公式（**以第一个查询位置对自己的注意力分数为例，后面都是这样**）：$$\text{scaled\_score}(Q_1,K_1)=\frac{score(Q_1,K_1)}{\sqrt{d_k}}$$  
缩放将对**整个注意力分数矩阵**进行缩放，注意力分数矩阵的每一行代表着一个“查询位置”（Query Position）对所有“键位置”（Key Positions）的注意力分数

**进行softmax操作:**

对所有缩放后的分数应用Softmax函数，得到**注意力权重**。  
$$
\text{attention\_weights}=softmax(\text{scaled\_score}(Q_1,K_1))$$  
这个同样也是对**整个注意力分数矩阵**进行操作。

**计算注意力输出：**